In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

rdd=sc.textFile("/home/ercan/Desktop/hadoop/datasets/DolarVerileri.txt")
rdd = rdd.map(lambda line : line.replace(".","").replace(",","."))
rdd = rdd.filter(lambda line : len(line.split("\t")[2]) > 0)

def splitLine(line):
    x = line.split("\t")
    return (x[1],float(x[2]))
rdd = rdd.map(lambda line : splitLine(line) )
rdd1 = rdd.zipWithIndex().map(lambda x : (x[1],x[0]))
rdd2 = rdd1.map(lambda x: (x[0]+1,x[1]))

joinedRDD = rdd1.join(rdd2).sortBy(lambda x: x[0])

def calculateDiffAndPer(item):
    prevDay = item[1][0][1]
    curDay = item[1][1][1]
    dates = item[1][0][0]+"-"+item[1][1][0]
    percentage = (curDay-prevDay)/prevDay
    return (dates,percentage)

joinedRDD=joinedRDD.map(lambda x : calculateDiffAndPer(x))
joinedRDD=joinedRDD.sortBy(lambda x :x[1],ascending=False)

print(joinedRDD.take(5))




[('03-01-2005-31-12-2004', 1001566.1641791044), ('27-02-2001-26-02-2001', 0.13387001667536716), ('31-10-2008-30-10-2008', 0.12666666666666662), ('25-03-1991-22-03-1991', 0.122293930234458), ('11-04-1994-08-04-1994', 0.09863817067204128)]
